# Mode solver Fully Vectorial

## Strip waveguides 

Lets compute the modes for different types of waveguides

```
    _________________________________


                                    clad_thickness
           width
         <---------->
          ___________    _ _ _ _ _ _
         |           |
    _____|           |____          |
                                    wg_heigth
    slab_thickness                     |
    _______________________ _ _ _ _ __

    sub_thickness
    _________________________________
    <------------------------------->
                 sub_width
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import modes as ms

In [ ]:
ms.waveguide(width=0.5, slab_thickness=0)

In [ ]:
s = ms.mode_solver_full(plot=True, n_modes=1, width=0.5, thickness=0.22, fields_to_write=('Ex'))

We can also plot the mode in logscale

In [ ]:
s = ms.mode_solver_full(plot=True, logscale=True, n_modes=1, width=0.5, thickness=0.22, fields_to_write=('Ex'))

In [ ]:
s = ms.mode_solver_full(plot=True) # by default it will plot at the fields

In [ ]:
ms.mode_solver_full?

where the waveguide parameters

In [ ]:
s.results['n_effs'][0].real

In [ ]:
s = ms.mode_solver_full(plot=False, n_modes=1, width=0.5, thickness=0.22, fields_to_write=('Ex'), overwrite=True)
A, centre, sigma_2 = ms.fit.fit_gaussian(s.wg.xc, s.wg.yc, np.abs(s.modes[0].fields['Ex']))
print(f"Aeff = {A:.2f}, mode_center = {centre}, Sigma2 = ({sigma_2[0]:.2f}, {sigma_2[1]:.2f})")

What is the waveguide geometry that gives the smallest mode area, for the TE polarization at 1550, for a silicon thickness of 220 nm, strip waveguide?

In [ ]:
widths = np.arange(200, 500, 20)*1e-3
aeffs = []
sigma2x = []
sigma2y = []

for width in widths:
    s = ms.mode_solver_full(plot=False, logscale=True, sub_width=4, n_modes=1, width=width, thickness=0.22, fields_to_write=('Ex'), clad_thickness=[1], sub_thickness=1, overwrite=True)
    A, centre, sigma2 = ms.fit.fit_gaussian(s.wg.xc, s.wg.yc, np.abs(s.modes[0].fields['Ex'])) 
    aeffs.append(A)
    sigma2x.append(sigma2[0])
    sigma2y.append(sigma2[1])

In [ ]:
plt.plot(widths, aeffs, 'o')
plt.xlabel('wg width (nm)')
plt.ylabel(r'Aeff ($\mu m^2$)')

plt.figure()
plt.plot(widths, sigma2x)
plt.xlabel('wg width (nm)')
plt.ylabel(r'$\sigma_x^2$')

plt.figure()
plt.plot(widths, sigma2y)
plt.xlabel('wg width (nm)')
plt.ylabel(r'$\sigma_y^2$')

As you can see 350 nm width will give you the smallest mode area for 220nm Silicon strip waveguides


## Rib waveguides

In [ ]:
s = ms.mode_solver_full(plot=True, n_modes=1, width=0.5, thickness=0.22, slab_thickness=0.09, fields_to_write=('Ex'), overwrite=True)

## Convergence tests

Lets do some convergence tests wherer we reduce the meshsize (20 nm by default)

In [ ]:
# dx=np.linspace(0.01, 0.02, 3)
dx = np.array([5, 10, 20, 40])*1e-3
n_effs = [ms.mode_solver_full(n_modes=1, x_step=dxi, y_step=dxi, plot=False).results['n_effs'][0].real for dxi in dx]

In [ ]:
plt.plot(dx*1e3, n_effs)
plt.xlabel('mesh size (nm)')
plt.ylabel('neff0')